In [ ]:
import os
from datetime import date, datetime
import yaml
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from experiment.toolkits.plotting_helper import *

project_dir = Path().resolve().parents[0]
sys.path.insert(0, str(project_dir))
import pickle
import datetime

In [ ]:
home = os.path.expanduser("~") 
os.chdir(str(project_dir))

# Waveform Loading Timing Benchmark

In [ ]:
base_cmd = "./bin/modules/wfm-streaming/timing-test"
def get_write_time (wfps, cb, nr):
    if (cb):
        command = base_cmd + " -wfps " + str(wfps) + " -cb" + " -nr" + str(nr)
    else:
        command = base_cmd + " -wfps " + str(wfps) + " -nr" + str(nr)
    return os.popen(command).read()

In [ ]:
reps = 1000
waveforms_per_segment = [1,2,4,8,16,32,64,128] 
continous_buffer = False
time_data  = []

for i in range(len(waveforms_per_segment)):
    print(f"{i}/{len(waveforms_per_segment)}")
    write_time_list = []
    wfps = waveforms_per_segment[i]
    write_time_str = get_write_time(wfps,
                                    continous_buffer, reps)
    time_data += tuple(float(write_time_str.split("\n")[0]), float(write_time_list.split("\n")[1]))

# Save results

In [ ]:
# Saving the results 
file_name = datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(home, "LLRS", "resources", "wfm-streaming-characterization", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": waveforms_per_segment,
            "experiment": file_name,
            "data": time_data
        }, file
        )

# Load results

In [ ]:
load_latest = True
if not load_latest:
    file_name = "2024-04-22, 22-49-39.pickle"

with open(os.path.join(home, "LLRS", "resources", "wfm-streaming-characterization", file_name), "rb") as file:
    data = pickle.load(file)
    roi_heights = np.array(data["xvals"])
    time_data = data["data"]
    title = data["experiment"]

In [ ]:
write_time_mean = time_data.mean(axis = 1) * 1000000000
write_time_std = time_data.std(axis = 1) 

fig, ax = getStylishFigureAxes(1, 1, dpi=300)

ErrorbarPlotter(
    fig,
    ax,
    roi_heights,
    write_time_mean / 1e6 ,
    yerr = write_time_std / 1e6 ,
    style = {"marker": "*", "linestyle": "None", "markersize": 1},
    xlim = [-50, 1074],
    ylim = [-1, 30],
    xticks = [0, 256, 512, 768, 1024],
    yticks = [0, 5, 10, 15, 20, 25, 30],
    xlabel = "Waveforms per segment",
    ylabel = "Loading time (microseconds)",
).draw()


plt.suptitle(f"{title.split('.')[0]}\nTransfer Buffer Loading Characterization", fontsize=7)
plt.tight_layout()
plt.savefig("result.jpg")
plt.show()

# Waveform Streaming Null Benchmark

In [ ]:
base_cmd = "./bin/modules/wfm-streaming/null-test"
def get_write_time (wfps, nm, nr):
    command = base_cmd + " " + str(nm) + " " + str(nwps) + " " + str(nr)
    return os.popen(command).read()

In [ ]:
reps = 1000
waveforms_per_segment = [1,2,4,8,16,32,64,128] 
number_of_moves = [16, 32, 64, 128, 256, 512, 1024, 1472, 2048]
time_data = []

for i in range(len(waveforms_per_segment)):
    print(f"{i}/{len(waveforms_per_segment)}")
    write_time_list = []
    wfps = waveforms_per_segment[i]
    temp_data = []
    for nm in range(number_of_moves):
        write_time_str = get_write_time(wfps, nm,reps)
        temp_data.append(float(write_time_str))
    time_data += temp_data

# Save results

In [ ]:
# Saving the results 
file_name = datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(home, "LLRS", "resources", "wfm-streaming-characterization", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": waveforms_per_segment,
            "experiment": file_name,
            "data": time_data
        }, file
        )

# Load results

In [ ]:
load_latest = True
if not load_latest:
    file_name = "2024-04-22, 22-49-39.pickle"

with open(os.path.join(home, "LLRS", "resources", "wfm-streaming-characterization", file_name), "rb") as file:
    data = pickle.load(file)
    roi_heights = np.array(data["xvals"])
    time_data = data["data"]
    title = data["experiment"]